In [ ]:
import pygame
import random
import sys
import os

# Constants
SIZE = 4
TILE_SIZE = 100
MARGIN = 10
WIDTH = HEIGHT = SIZE * (TILE_SIZE + MARGIN) + MARGIN + 100
BACKGROUND_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180), 2: (238, 228, 218), 4: (237, 224, 200), 8: (242, 177, 121),
    16: (245, 149, 99), 32: (246, 124, 95), 64: (246, 94, 59),
    128: (237, 207, 114), 256: (237, 204, 97), 512: (237, 200, 80),
    1024: (237, 197, 63), 2048: (237, 194, 46)
}
FONT_COLOR = (119, 110, 101)
SCORE_COLOR = (255, 255, 255)
UNDO_STACK_SIZE = 5

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2048 Game")
font = pygame.font.Font(None, 36)
big_font = pygame.font.Font(None, 72)

def draw_board(board, score, high_score):
    screen.fill(BACKGROUND_COLOR)
    for row in range(SIZE):
        for col in range(SIZE):
            value = board[row][col]
            pygame.draw.rect(screen, TILE_COLORS.get(value, (60, 58, 50)),
                             (col * (TILE_SIZE + MARGIN) + MARGIN,
                              row * (TILE_SIZE + MARGIN) + MARGIN,
                              TILE_SIZE, TILE_SIZE))
            if value:
                label = big_font.render(str(value), True, FONT_COLOR)
                label_rect = label.get_rect(center=(col * (TILE_SIZE + MARGIN) + MARGIN + TILE_SIZE // 2,
                                                    row * (TILE_SIZE + MARGIN) + MARGIN + TILE_SIZE // 2))
                screen.blit(label, label_rect)
    score_label = font.render(f"Score: {score}", True, SCORE_COLOR)
    high_score_label = font.render(f"High Score: {high_score}", True, SCORE_COLOR)
    screen.blit(score_label, (MARGIN, HEIGHT - 70))
    screen.blit(high_score_label, (MARGIN, HEIGHT - 40))
    pygame.display.flip()

def add_new_tile(board):
    empty_tiles = [(r, c) for r in range(SIZE) for c in range(SIZE) if board[r][c] == 0]
    if not empty_tiles:
        return
    row, col = random.choice(empty_tiles)
    board[row][col] = 2 if random.random() < 0.9 else 4

def merge(row):
    new_row = [i for i in row if i != 0]
    score_increment = 0
    if len(new_row) <= 1:
        return new_row + [0] * (SIZE - len(new_row)), score_increment
    for i in range(len(new_row) - 1):
        if new_row[i] == new_row[i + 1]:
            new_row[i] *= 2
            score_increment += new_row[i]
            new_row[i + 1] = 0
    new_row = [i for i in new_row if i != 0]
    return new_row + [0] * (SIZE - len(new_row)), score_increment

def move(board, direction):
    moved = False
    score_increment = 0
    if direction == 'left':
        for row in range(SIZE):
            new_row, incr = merge(board[row])
            if board[row] != new_row:
                moved = True
            board[row] = new_row
            score_increment += incr
    elif direction == 'right':
        for row in range(SIZE):
            new_row, incr = merge(board[row][::-1])
            new_row = new_row[::-1]
            if board[row] != new_row:
                moved = True
            board[row] = new_row
            score_increment += incr
    elif direction == 'up':
        for col in range(SIZE):
            column = [board[row][col] for row in range(SIZE)]
            new_col, incr = merge(column)
            for row in range(SIZE):
                if board[row][col] != new_col[row]:
                    moved = True
                board[row][col] = new_col[row]
            score_increment += incr
    elif direction == 'down':
        for col in range(SIZE):
            column = [board[row][col] for row in range(SIZE)][::-1]
            new_col, incr = merge(column)
            new_col = new_col[::-1]
            for row in range(SIZE):
                if board[row][col] != new_col[row]:
                    moved = True
                board[row][col] = new_col[row]
            score_increment += incr
    return moved, score_increment

def save_high_score(score):
    high_score = load_high_score()
    if score > high_score:
        with open("high_score.txt", "w") as f:
            f.write(str(score))

def load_high_score():
    if not os.path.exists("high_score.txt"):
        return 0
    with open("high_score.txt", "r") as f:
        return int(f.read())

def main():
    board = [[0] * SIZE for _ in range(SIZE)]
    add_new_tile(board)
    add_new_tile(board)
    score = 0
    high_score = load_high_score()
    undo_stack = []
    difficulty = "normal"  # Can be "easy", "normal", or "hard"

    if difficulty == "easy":
        board[random.randint(0, SIZE - 1)][random.randint(0, SIZE - 1)] = 4
    elif difficulty == "hard":
        board[random.randint(0, SIZE - 1)][random.randint(0, SIZE - 1)] = 8

    draw_board(board, score, high_score)  # Initial draw

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                save_high_score(score)
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                moved = False
                if event.key == pygame.K_LEFT:
                    moved, incr = move(board, 'left')
                elif event.key == pygame.K_RIGHT:
                    moved, incr = move(board, 'right')
                elif event.key == pygame.K_UP:
                    moved, incr = move(board, 'up')
                elif event.key == pygame.K_DOWN:
                    moved, incr = move(board, 'down')
                elif event.key == pygame.K_u and undo_stack:
                    board, score = undo_stack.pop()
                    moved = False
                else:
                    continue

                if moved:
                    score += incr
                    if len(undo_stack) >= UNDO_STACK_SIZE:
                        undo_stack.pop(0)
                    undo_stack.append(([[board[row][col] for col in range(SIZE)] for row in range(SIZE)], score))
                    add_new_tile(board)
                    high_score = max(high_score, score)
                    draw_board(board, score, high_score)

if __name__ == "__main__":
    main()
